In [ ]:
import numpy as np
from PIL import Image
import time, re, os, sys
import matplotlib.pyplot as plt

In [ ]:
FOLDER = ''
POINT_NUM = 10000
LENGTH = 100
CLUSTER_NUM = 2
colormap = ['black', 'darkorange', 'cornflowerblue', 'silver', 
	'gold', 'green', 'navy', 'magenta', 'yellow', 'red', 'peru', 'pink', 
	'dodgerblue', 'greenyellow', 'cyan']

In [ ]:
def openImage(filename):
	img = Image.open(filename)
	img = np.array(img)
	return img